In [217]:
import requests
from key import fred_key
import pandas as pd
import numpy as np

base_url = "https://api.stlouisfed.org/fred/"
obs_endpoint = "series/observations"

start_date = "1950-01-01"
end_date = "2023-12-31"



d_gdp = {
    'name': 'GDP',
    'series_id': "GDPC1",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_cnd = {
    'name': 'CND',
    'series_id': "PCEND",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_cd = {
    'name': 'CD',
    'series_id': "PCEDG",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}


d_h = {
    'name': 'H',
    'series_id': "HOANBS",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

# excluding farm employees
d_l = {
    'name': 'L',
    'series_id': "PAYEMS",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_AveW = {
    'name': 'AveW',
    'series_id': "AHETPI",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

request_parameters = [d_gdp, d_cd, d_cnd, d_h, d_l, d_AveW]




In [219]:
def fetch_data(parameter):
    response = requests.get(base_url + obs_endpoint, params=parameter)
    if response.status_code == 200:
        res_data = response.json()
        obs_data = pd.DataFrame(res_data['observations'])
        obs_data['date'] = pd.to_datetime(obs_data['date'])
        obs_data.set_index('date', inplace=True)
        obs_data = obs_data.drop(['realtime_start', 'realtime_end'], axis=1)

        # Convert 'value' to numeric, coercing errors to NaN
        obs_data['value'] = pd.to_numeric(obs_data['value'], errors='coerce')

        obs_data.rename(columns={'value': parameter['name']}, inplace=True)
        return obs_data
    else:
        print('Failed to retrieve data. Status code:', response.status_code)
        return pd.DataFrame()  # Return an empty DataFrame on failure
    
    
dataframe = pd.DataFrame()
# concatenate the data
for parameter in request_parameters:
    df = fetch_data(parameter)
    if dataframe.empty:
        dataframe = df
    else:
        dataframe = pd.concat([dataframe, df], axis=1)

# Display the merged dataframe
dataframe

,GDP,CD,CND,H,L,AveW
date,,,,,,
1950-01-01,2458.532,NaN,NaN,42.265,45282,NaN
1951-01-01,2656.320,NaN,NaN,44.261,47926,NaN
1952-01-01,2764.803,NaN,NaN,44.753,48903,NaN
1953-01-01,2894.412,NaN,NaN,45.880,50304,NaN
1954-01-01,2877.708,NaN,NaN,44.331,49087,NaN
...,...,...,...,...,...,...
2019-01-01,20692.087,1522.7,3006.5,102.791,150906,23.51
2020-01-01,20234.074,1628.9,3084.2,95.064,142165,24.69
2021-01-01,21407.693,2006.4,3500.2,100.227,146276,25.91


In [224]:
# adjust the dataset

# add productivity
dataframe['GDP/L'] = (dataframe['GDP']/dataframe['L'])  # annual average income
dataframe['AveH'] = dataframe['H']/dataframe['L']*1000
dataset1 = dataframe.dropna()
# data from 1964 to 2022
dataset1



,GDP,CD,CND,H,L,AveW,GDP/L,AveH
date,,,,,,,,
1964-01-01,4205.277,59.5,152.7,50.351,58394,2.54,0.072016,0.862263
1965-01-01,4478.555,66.4,163.3,52.297,60879,2.63,0.073565,0.859032
1966-01-01,4773.931,71.8,177.9,54.107,64025,2.73,0.074564,0.845092
1967-01-01,4904.864,74.0,185.0,54.113,65935,2.86,0.074389,0.820702
1968-01-01,5145.914,84.8,199.8,55.074,68027,3.02,0.075645,0.809590
1969-01-01,5306.595,90.5,214.2,56.666,70515,3.22,0.075255,0.803602
1970-01-01,5316.391,90.0,228.8,55.799,71007,3.40,0.074871,0.785824
1971-01-01,5491.446,102.4,239.7,55.704,71331,3.63,0.076985,0.780923
1972-01-01,5780.048,116.4,257.4,57.455,73788,3.90,0.078333,0.778650


In [226]:

sd = dataset1.std()/dataset1.mean()

# Define lags
lags = range(-4, 5)

# Initialize a DataFrame to store cross-correlation values
cross_corr_df = pd.DataFrame(index=dataset1.columns, columns=lags)

# Calculate cross-correlation for each variable and each lag
for col in dataset1.columns:
    for lag in lags:
        cross_corr_df.at[col, lag] = dataset1['GDP'].corr(dataset1[col].shift(-lag))



cross_corr_df['SD%'] = sd
table1 = cross_corr_df
table1


,-4,-3,-2,-1,0,1,2,3,4,SD%
GDP,0.996157,0.996902,0.998006,0.999012,1.0,0.999012,0.998006,0.996902,0.996157,0.456230
CD,0.991568,0.993298,0.989355,0.985254,0.981415,0.978577,0.976664,0.974612,0.974694,0.781369
CND,0.989164,0.990058,0.989764,0.987779,0.986553,0.986209,0.98596,0.98637,0.988491,0.750827
H,0.956995,0.951185,0.950265,0.951792,0.953955,0.948412,0.940684,0.933495,0.926917,0.206383
L,0.975264,0.971531,0.969728,0.970067,0.971627,0.9699,0.966303,0.962447,0.958619,0.255275
AveW,0.995252,0.995148,0.994857,0.994162,0.993219,0.993012,0.992738,0.992388,0.993354,0.583056
GDP/L,0.988295,0.990481,0.99224,0.99373,0.99417,0.994016,0.995472,0.996699,0.997268,0.226308
AveH,-0.881602,-0.883717,-0.879279,-0.875903,-0.876424,-0.889888,-0.908078,-0.924259,-0.930663,0.065301


In [227]:
series_ids = {
    'PerCapitaOutput': 'A939RC0Q052SBEA',
    'PerCapitaConsumption': 'A794RC0Q052SBEA',
    'RealWage': 'LES1252881600Q',
    'RealInterestRate': 'FEDFUNDS',
    'TotalFactorProductivity': 'TFP'
}